In [1]:
import numpy as np
import pandas as pd

def read_lammpstrj(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    data = []
    timestep = None
    num_atoms = None
    box_bounds = None
    atom_data = []

    for line in lines:
        if "ITEM: TIMESTEP" in line:
            if atom_data:
                data.append((timestep, np.array(atom_data)))
                atom_data = []
            timestep = int(lines[lines.index(line) + 1].strip())
        elif "ITEM: NUMBER OF ATOMS" in line:
            num_atoms = int(lines[lines.index(line) + 1].strip())
        elif "ITEM: BOX BOUNDS" in line:
            box_bounds = [
                list(map(float, lines[lines.index(line) + i + 1].strip().split()))
                for i in range(3)
            ]
        elif "ITEM: ATOMS" in line:
            columns = line.strip().split()[2:]
            for atom_line in lines[lines.index(line) + 1: lines.index(line) + 1 + num_atoms]:
                atom_data.append(list(map(float, atom_line.strip().split())))
    
    if atom_data:
        data.append((timestep, np.array(atom_data)))

    return data, columns

data, columns = read_lammpstrj('dump.lammpstrj')


In [2]:
def compute_neighbors(positions, cutoff):
    num_atoms = len(positions)
    neighbors = {i: [] for i in range(num_atoms)}
    for i in range(num_atoms):
        for j in range(i + 1, num_atoms):
            distance = np.linalg.norm(positions[i, 1:4] - positions[j, 1:4])
            if distance <= cutoff:
                neighbors[i].append(j)
                neighbors[j].append(i)
    return neighbors

cutoff_distance = 1.0  # Define your cutoff distance here

all_neighbors = []
for timestep, positions in data:
    neighbors = compute_neighbors(positions, cutoff_distance)
    all_neighbors.append((timestep, neighbors))


KeyboardInterrupt: 

In [5]:
import networkx as nx

def create_network(neighbors):
    G = nx.Graph()
    for i, neighs in neighbors.items():
        for j in neighs:
            G.add_edge(i, j)
    return G

for timestep, neighbors in all_neighbors:
    G = create_network(neighbors)
    degrees = dict(G.degree())
    clustering_coeffs = nx.clustering(G)
    shortest_paths = dict(nx.all_pairs_shortest_path_length(G))
    
    # You can store or process these metrics as needed
    print(f"Timestep: {timestep}")
    print(f"Degrees: {degrees}")
    print(f"Clustering Coefficients: {clustering_coeffs}")
    print(f"Shortest Paths: {shortest_paths}")


AttributeError: 'csr_matrix' object has no attribute 'items'

In [6]:
import matplotlib.pyplot as plt

timestep_to_visualize = 0  # Change this to visualize a different timestep

for timestep, neighbors in all_neighbors:
    if timestep == timestep_to_visualize:
        G = create_network(neighbors)
        pos = nx.spring_layout(G)
        nx.draw(G, pos, with_labels=True, node_size=500, node_color="lightblue", edge_color="gray")
        plt.title(f"Network at Timestep {timestep}")
        plt.show()
        break


In [7]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from scipy.sparse import csr_matrix

def read_lammpstrj(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    data = []
    atom_data = []
    timestep = None
    num_atoms = None

    for i, line in enumerate(lines):
        if "ITEM: TIMESTEP" in line:
            if atom_data:
                data.append((timestep, np.array(atom_data)))
                atom_data = []
            timestep = int(lines[i + 1].strip())
        elif "ITEM: NUMBER OF ATOMS" in line:
            num_atoms = int(lines[i + 1].strip())
        elif "ITEM: ATOMS" in line:
            for atom_line in lines[i + 1: i + 1 + num_atoms]:
                atom_data.append(list(map(float, atom_line.strip().split())))
    
    if atom_data:
        data.append((timestep, np.array(atom_data)))

    return data

def compute_neighbors(positions, cutoff):
    dist_matrix = squareform(pdist(positions[:, 1:4]))
    neighbors = (dist_matrix <= cutoff) & (dist_matrix > 0)
    sparse_neighbors = csr_matrix(neighbors)
    return sparse_neighbors

def create_network(sparse_neighbors):
    G = nx.from_scipy_sparse_matrix(sparse_neighbors)
    return G

data = read_lammpstrj('dump.lammpstrj')
cutoff_distance = 1.0  # Define your cutoff distance here

all_neighbors = []
for timestep, positions in data:
    sparse_neighbors = compute_neighbors(positions, cutoff_distance)
    all_neighbors.append((timestep, sparse_neighbors))

# Analyze and visualize the network
for timestep, sparse_neighbors in all_neighbors:
    G = create_network(sparse_neighbors)
    degrees = dict(G.degree())
    clustering_coeffs = nx.clustering(G)
    shortest_paths = dict(nx.all_pairs_shortest_path_length(G))

    # Print or process these metrics as needed
    print(f"Timestep: {timestep}")
    print(f"Degrees: {degrees}")
    print(f"Clustering Coefficients: {clustering_coeffs}")
    print(f"Shortest Paths: {shortest_paths}")

# Visualization
timestep_to_visualize = 0  # Change this to visualize a different timestep

for timestep, sparse_neighbors in all_neighbors:
    if timestep == timestep_to_visualize:
        G = create_network(sparse_neighbors)
        pos = nx.spring_layout(G)
        nx.draw(G, pos, with_labels=True, node_size=500, node_color="lightblue", edge_color="gray")
        plt.title(f"Network at Timestep {timestep}")
        plt.show()
        break


AttributeError: module 'networkx' has no attribute 'from_scipy_sparse_matrix'